# EDA for Train.csv

# 1. 작업 환경 설정 및 데이터 구조 확인

In [ ]:
!pip install hvplot

In [ ]:
import pandas as pd
import hvplot
import hvplot.pandas

In [ ]:
# data load
train = pd.read_csv('../input/train.csv')

In [ ]:
# data 정보 확인
train.info()

In [ ]:
# 결측치 개수 확인
train.isnull().sum()

## 2. first_active_month
- `first_active_month`는 '처음으로 구매가 발생한 달'을 의미한다.

### 2.1 시간에 따른 first_active_month의 분포

In [ ]:
# 각 first_active_month의 개수 확인
first_active_month_count = pd.DataFrame(train['first_active_month'].value_counts()).reset_index()
first_active_month_count.rename(columns = {"first_active_month":'count', 'index':'first_active_month'}, inplace=True)

# first_active_month를 기준으로 오름차순으로 정렬 후 시각화
sorted_first_active_month_count = first_active_month_count.sort_values('first_active_month')
(sorted_first_active_month_count.hvplot.line(x='first_active_month', y='count', rot=90, width=800, title="시간에 따른 first_active_month의 분포") +
sorted_first_active_month_count.hvplot.table(width=300))

- 전체적으로 보았을 때 지속적으로 증가하다 2017년 9월 정점을 찍은 후 급격히 감소했다.
- 특정 시점에서 갑작스럽게 증가하거나 감소하는 등 급격한 변화가 있었다.
- first_active_month의 증가는 완벽하게 일치하진 않지만, 사용자의 수가 그만큼 증가했다고도 해석할 수 있다. 따라서 first_active_month의 count의 증가는 Elo의 사용자 수가 그만큼 증가했다고도 해석할 수 있다. 또한, 2018년 1월, 2월에 급격하게 하락한 이유 또한 Elo의 사용자 수와 연관 있을 것이라 추측할 수 있다.

In [ ]:
# Q. 직전달과 비교했을 때 변화율이 가장 높은 구간은?
sorted_first_active_month_count['pct_change'] = sorted_first_active_month_count['count'].pct_change()
sorted_first_active_month_count.sort_values('pct_change').head()

- 직전달과 비교했을 때 2018년 1월과 2월의 변화율이 각각 -99%, -98%로 가장 컸다.
- 이후의 EDA 과정에서 왜 이런 결과가 발생했는지 확인해 볼 필요성이 있어 보인다.

### 2.2 시간에 따른 충성도의 변화

In [ ]:
grouped_first_active_month = train.groupby('first_active_month').agg({'target':'mean'}).sort_values('first_active_month').reset_index()
(grouped_first_active_month.hvplot.line(x='first_active_month', y='target', rot=90, title="시간에 따른 충성도의 변화") +
grouped_first_active_month.hvplot.table(width=300))

### 2.3 `target` 변수의 결측치
- `target` 변수와 다른 변수와의 관계를 보던 중 특이한 값(-33.21928095)을 발견했다.
- 결측치로 의심이 되어 해당 값을 결측치로 볼 수 있는지 확인해보기로 했다.

#### 2.3.1 `feature_1, 2, 3` 세 변수의 모든 카테고리 레벨에서 해당 값이 등장했다.

In [ ]:
(train.sort_values('feature_1').hvplot.box('target', by='feature_1' ,subplots=True, shared_axes=False, height=250, width=500) +
train.sort_values('feature_2').hvplot.box('target', by='feature_2' ,subplots=True, shared_axes=False, height=250, width=500) +
train.sort_values('feature_3').hvplot.box('target', by='feature_3' ,subplots=True, shared_axes=False, height=250, width=500)).cols(2)

- boxplot을 그려본 결과, `feature_1`, `feature_2`, `feature_3`의 카테고리 레벨별 충성도의 평균은 모두 0과 근접했다.
- 또한, 소수점 8자리까지 정확히 일치하는 특정 값(-33.21928095)이 세 변수, 모든 카테고리 레벨에서 등장했다.

In [ ]:
train['target'].min()

- pandas dataframe에선 기본값으로 소수점 표시 제한이 있어서 정확한 수치가 표시되지 않는다.
- 해당 값의 정확한 값은 -33.21928095이다.

#### 2.3.2 해당값이 지나치게 많다.

In [ ]:
print(train['target'].value_counts().head())

print('\n해당 row의 개수 : {:,}개'.format(len(train[train['target'] == -33.21928095])))
print('전체 row에서 해당 값의 비율 : {}%'.format(round((len(train[train['target'] == -33.21928095]) / len(train)) * 100, 2)))

- 해당 값은 `target` 값 별 개수는 2,207개로, 가장 많은 부분(1%)을 차지한다.
- 충성도가 0인 고객들을 제외하고 본다면, 비정상적으로 등장 빈도가 높은 값이다.

In [ ]:
train[train['target'] > -33.21928095].sort_values('target').head()

- `target` 변수를 오름차순으로 정렬하여 해당 값(-33.21928095)보다 큰 바로 위의 수를 확인했다.
- -17.608147으로 해당 값과는 큰 차이를 보였다.

#### 2.3.3 해당값을 제외한 다른 값들의 범위와 지나치게 차이가 난다.

In [ ]:
print(train.sort_values('target', ascending=False).head())
print(train[train['target'] > -33.21928095].sort_values('target').head())

- 해당 값(-33.21928095)을 제외하면,`target` 변수의 값의 범위는 -17 ~ 18이다.

- 종합해 보자면, 아래의 세 가지 이유로 해당 값(-33.21928095)을 결측치로 볼 수 있다.
    1. 소수점 8자리까지 정확하게 일치하는 값이 `feature_1, 2, 3` 세 변수의 모든 카테고리 레벨에서 등장한다.
    2. 다른 값들과 비교했을 때, 해당 값의 개수가 지나치게 많다.
    3. 해당 값을 제외한 다른 값들의 범위(-17 ~ 18)와 지나치게 차이가 많이 난다.
- 또한, 경험적인 이유에서도 해당 값이 결측치로 의심스럽다.
    1. 현업에선 결측치를 NaN으로 넣는 것보다 미리 협의된 특정값으로 넣는 게 편하기 때문에 이러한 방식을 많이 쓴다고 알고 있다.
    2. Kaggle의 Competetion이었던 KKBox's Music Recommendation Challenge에서도 사용자가 입력하지 않고 Skip한 값을 결측치가 아닌 -1로 표시했다.
- 해당 변수(`target`)가 충성도를 의미한다는 것을 생각한다면, 아마 충성도를 계산할 수 없을 정도로 정보가 적은 고객들은 충성도 값을 결측치 대신 해당 값(-33.21928095)으로 넣은 것이라 추측했다. 이후의 EDA를 통해 이를 확인해 볼 예정이다.